# Spesifikasi

MP = 5

Electrolytic Capacitor: 25 V Nippon KZE series

# Single Capacitor

47	100	220	330	470	680	820	1000	1500	1800	2200	2700

# Series

In [1]:
from itertools import combinations

def calculate_series_combinations(capacitor_values, max_parallel=5):
    series_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitor_values) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitor_values, r):
            # Calculate equivalent capacitance for the series combination
            equivalent_capacitance = sum(1 / capacitor for capacitor in combination) ** -1

            # Append the combination and its equivalent capacitance to the list
            series_combinations.append({
                'combination': combination,
                'equivalent_capacitance': equivalent_capacitance
            })

    return series_combinations

# Given list of capacitor values
capacitor_values = [47, 100, 220, 330, 470, 680, 820, 1000, 1500, 1800, 2200, 2700]

# Maximum parallel connections (MP)
max_parallel = 5

# Calculate series combinations
series_combinations = calculate_series_combinations(capacitor_values, max_parallel)

# Print the results
for combination in series_combinations:
    print(f"Combination: {combination['combination']}, Equivalent Capacitance: {combination['equivalent_capacitance']} μF")


Combination: (47,), Equivalent Capacitance: 47.0 μF
Combination: (100,), Equivalent Capacitance: 100.0 μF
Combination: (220,), Equivalent Capacitance: 220.0 μF
Combination: (330,), Equivalent Capacitance: 330.0 μF
Combination: (470,), Equivalent Capacitance: 470.0 μF
Combination: (680,), Equivalent Capacitance: 680.0 μF
Combination: (820,), Equivalent Capacitance: 820.0 μF
Combination: (1000,), Equivalent Capacitance: 1000.0 μF
Combination: (1500,), Equivalent Capacitance: 1500.0 μF
Combination: (1800,), Equivalent Capacitance: 1800.0 μF
Combination: (2200,), Equivalent Capacitance: 2200.0 μF
Combination: (2700,), Equivalent Capacitance: 2700.0 μF
Combination: (47, 100), Equivalent Capacitance: 31.97278911564626 μF
Combination: (47, 220), Equivalent Capacitance: 38.72659176029963 μF
Combination: (47, 330), Equivalent Capacitance: 41.14058355437666 μF
Combination: (47, 470), Equivalent Capacitance: 42.72727272727273 μF
Combination: (47, 680), Equivalent Capacitance: 43.96148555708391 μF

In [3]:
import csv
from itertools import combinations

def calculate_series_combinations(capacitor_values, max_parallel=5):
    series_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitor_values) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitor_values, r):
            # Calculate equivalent capacitance for the series combination
            equivalent_capacitance = sum(1 / capacitor for capacitor in combination) ** -1

            # Append the combination and its equivalent capacitance to the list
            series_combinations.append({
                'combination': combination,
                'equivalent_capacitance': equivalent_capacitance
            })

    return series_combinations

def save_to_csv(series_combinations, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Combination', 'Equivalent Capacitance (μF)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for combination in series_combinations:
            writer.writerow({
                'Combination': str(combination['combination']),
                'Equivalent Capacitance (μF)': round(combination['equivalent_capacitance'], 2)
            })


# Given list of capacitor values
capacitor_values = [47, 100, 220, 330, 470, 680, 820, 1000, 1500, 1800, 2200, 2700]

# Maximum parallel connections (MP)
max_parallel = 5

# Calculate series combinations
series_combinations = calculate_series_combinations(capacitor_values, max_parallel)

# Save to CSV file
csv_filename = 'series_combinations.csv'
save_to_csv(series_combinations, csv_filename)
print(f"Series combinations saved to {csv_filename}")


Series combinations saved to series_combinations.csv


In [4]:
import csv
from itertools import combinations

def read_capacitor_values_from_csv(csv_filename):
    capacitor_values = []

    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            capacitor_values.append(float(row['Capacitor']))

    return capacitor_values

def calculate_series_combinations(capacitor_values, max_parallel=5):
    series_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitor_values) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitor_values, r):
            # Calculate equivalent capacitance for the series combination
            equivalent_capacitance = sum(1 / capacitor for capacitor in combination) ** -1

            # Append the combination and its equivalent capacitance to the list
            series_combinations.append({
                'combination': combination,
                'equivalent_capacitance': equivalent_capacitance
            })

    return series_combinations

def save_to_csv(series_combinations, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Combination', 'Equivalent Capacitance (μF)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for combination in series_combinations:
            writer.writerow({
                'Combination': str(combination['combination']),
                'Equivalent Capacitance (μF)': round(combination['equivalent_capacitance'])
            })

# Specify the CSV file containing capacitor values
capacitor_csv_filename = 'capacitor.csv'

# Read capacitor values from CSV
capacitor_values = read_capacitor_values_from_csv(capacitor_csv_filename)

# Maximum parallel connections (MP)
max_parallel = 5

# Calculate series combinations
series_combinations = calculate_series_combinations(capacitor_values, max_parallel)

# Save to CSV file
csv_filename = 'series_combinations.csv'
save_to_csv(series_combinations, csv_filename)
print(f"Series combinations saved to {csv_filename}")


Series combinations saved to series_combinations.csv


In [7]:
import csv
from itertools import combinations

def read_capacitor_values_from_csv(csv_filename):
    capacitors = []

    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            capacitor = {
                'value': float(row['Capacitor']),
                'volume': float(row['Volume'])  # Add this line to read the volume
            }
            capacitors.append(capacitor)

    return capacitors

def calculate_series_combinations(capacitors, max_parallel=5):
    series_combinations = []

    for r in range(1, min(max_parallel + 1, len(capacitors) + 1)):
        # Generate all possible combinations of r capacitors
        for combination in combinations(capacitors, r):
            # Calculate equivalent capacitance for the series combination
            equivalent_capacitance = sum(1 / capacitor['value'] for capacitor in combination) ** -1

            # Calculate total volume for the series combination
            total_volume = sum(capacitor['volume'] for capacitor in combination)

            # Append the combination, equivalent capacitance, and total volume to the list
            series_combinations.append({
                'combination': [capacitor['value'] for capacitor in combination],
                'equivalent_capacitance': equivalent_capacitance,
                'total_volume': total_volume
            })

    return series_combinations

def save_to_csv(series_combinations, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Combination', 'Capacitance', 'Total Volume']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for combination in series_combinations:
            writer.writerow({
                'Combination': str(combination['combination']),
                'Capacitance': round(combination['equivalent_capacitance']),
                'Total Volume': round(combination['total_volume'], 2)
            })

# Specify the CSV file containing capacitor values and volumes
capacitor_csv_filename = 'capacitor.csv'

# Read capacitor values and volumes from CSV
capacitors = read_capacitor_values_from_csv(capacitor_csv_filename)

# Maximum parallel connections (MP)
max_parallel = 5

# Calculate series combinations
series_combinations = calculate_series_combinations(capacitors, max_parallel)

# Save to CSV file
csv_filename = 'series_combinations.csv'
save_to_csv(series_combinations, csv_filename)
print(f"Series combinations saved to {csv_filename}")


Series combinations saved to series_combinations.csv
